In [1]:
import requests
import pandas as pd
from io import BytesIO

def download_excel_to_dataframe(url, sheet_name=None):
    """
    Télécharge un fichier Excel et charge une feuille dans un DataFrame.
    
    Args:
        url (str): Lien du fichier Excel.
        sheet_name (str, optional): Feuille à charger (par défaut, la première).
        
    Returns:
        pd.DataFrame: Données de la feuille, ou None en cas d'erreur.
    """
    try:
        # Télécharger le fichier
        response = requests.get(url)
        response.raise_for_status()
        
        # Charger le fichier Excel
        file_content = BytesIO(response.content)
        excel_data = pd.ExcelFile(file_content)
        
        # Utiliser la première feuille si non spécifié
        if sheet_name is None:
            sheet_name = excel_data.sheet_names[0]
            print(f"Feuille non spécifiée, chargement de : {sheet_name}")
        
        # Charger la feuille dans un DataFrame
        df = excel_data.parse(sheet_name)
        print(f"Feuille '{sheet_name}' chargée.")
        
        return df
    except Exception as e:
        print(f"Erreur : {e}")
        return None

Erreur : 410 Client Error: Gone for url: http://ww38.exemple.com/fichier.xlsx


In [30]:
import requests
import zipfile
from io import BytesIO
import pandas as pd

def download_excel_to_dataframe(url, excel_file_name=None, sheet_name=None):
    """
    Télécharge un fichier Excel ou un fichier ZIP contenant un fichier Excel, 
    et charge une feuille dans un DataFrame.
    
    Args:
        url (str): Lien du fichier (Excel ou ZIP).
        excel_file_name (str, optional): Nom du fichier Excel à extraire du ZIP si nécessaire.
        sheet_name (str, optional): Feuille à charger (par défaut, la première).
        
    Returns:
        pd.DataFrame: Données de la feuille, ou None en cas d'erreur.
    """
    try:
        # Télécharger le fichier
        response = requests.get(url)
        response.raise_for_status()
        
        # Vérifier si c'est un fichier ZIP
        content_type = response.headers.get('Content-Type', '').lower()
        if 'zip' in content_type or url.endswith('.zip'):
            print("Le fichier téléchargé est un fichier ZIP.")
            zip_content = BytesIO(response.content)
            with zipfile.ZipFile(zip_content) as zip_file:
                # Si aucun nom de fichier Excel n'est spécifié, utiliser le premier fichier Excel trouvé
                if excel_file_name is None:
                    excel_file_name = next((f for f in zip_file.namelist() if f.endswith('.xlsx')), None)
                    if excel_file_name is None:
                        raise FileNotFoundError("Aucun fichier Excel (.xlsx) n'a été trouvé dans le ZIP.")
                    print(f"Fichier Excel non spécifié, utilisation de : {excel_file_name}")
                
                # Vérifier si le fichier Excel spécifié existe dans le ZIP
                if excel_file_name not in zip_file.namelist():
                    raise FileNotFoundError(f"Le fichier '{excel_file_name}' est introuvable dans le ZIP.")
                
                # Extraire le fichier Excel dans un objet BytesIO
                with zip_file.open(excel_file_name) as excel_file:
                    excel_data = pd.ExcelFile(BytesIO(excel_file.read()))  # Lire tout le fichier en mémoire
        
        # Si ce n'est pas un ZIP, on suppose que c'est un fichier Excel
        elif 'excel' in content_type or url.endswith('.xlsx'):
            print("Le fichier téléchargé est un fichier Excel.")
            excel_data = pd.ExcelFile(BytesIO(response.content))
        
        else:
            raise ValueError("Le fichier téléchargé n'est ni un fichier Excel, ni un fichier ZIP.")
        
        # Utiliser la première feuille si non spécifié
        if sheet_name is None:
            sheet_name = excel_data.sheet_names[0]
            print(f"Feuille non spécifiée, chargement de : {sheet_name}")
        
        # Charger la feuille dans un DataFrame
        df = excel_data.parse(sheet_name)
        print(f"Feuille '{sheet_name}' chargée.")
        
        return df
    
    except Exception as e:
        print(f"Erreur : {e}")
        return None


In [31]:
chômage = download_excel_to_dataframe("https://www.insee.fr/fr/statistiques/fichier/2532173/econ-gen-taux-cho-trim-2.xlsx")
chômage

Le fichier téléchargé est un fichier Excel.
Feuille non spécifiée, chargement de : Données
Feuille 'Données' chargée.


,Taux de chômage selon le sexe et l'âge,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en %,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sexe et âge,1975-T1,1975-T2,1975-T3,1975-T4,1976-T1,1976-T2,1976-T3,1976-T4,1977-T1,...,2022-T2,2022-T3,2022-T4,2023-T1,2023-T2,2023-T3,2023-T4,2024-T1,2024-T2,2024-T3
3,Femmes,4.6,4.9,5.2,5.5,5.8,5.9,5.8,5.8,6.1,...,7.2,7,6.9,6.8,7.1,7.4,7.4,7.3,7.4,7.2
4,15-24 ans,9.8,10.4,11.1,11.8,12.8,13.1,12.7,12.6,13.5,...,17.3,15.7,14.8,15.6,16,16,16.7,17,18.2,20
5,25-49 ans,3.3,3.5,3.7,3.9,4,4,4,4,4.1,...,6.6,6.4,6.4,6.3,6.6,7.2,7.1,7,6.8,6.3
6,50 ans ou plus,2.1,2.2,2.4,2.5,2.6,2.7,2.8,3,3.2,...,5,5.2,5.2,5,5.1,5.2,4.9,4.9,5,4.8
7,Hommes,2.3,2.6,2.8,2.9,2.9,2.8,2.7,2.7,2.9,...,7.6,7.5,7.3,7.4,7.4,7.4,7.7,7.6,7.3,7.6
8,15-24 ans,4.8,5.4,5.8,6,6,5.8,5.6,5.9,6.6,...,18.9,18.9,18.7,17.8,18.1,18.4,18.3,19.3,17.7,19.4
9,25-49 ans,1.7,1.9,2.1,2.1,2.1,2,1.9,1.9,2,...,6.8,6.5,6.5,6.4,6.6,6.4,6.9,6.6,6.5,6.9


In [32]:
chômage.columns = chômage.iloc[2] 
chômage = chômage.drop(index=list(range(0, 2)) + list(range(3, 11)) + list(range(12, 21)))
chômage = chômage.drop(index=2) 
chômage.rename(columns={"Sexe et âge": "TrimAnnee"}, inplace=True)
chômage

2,TrimAnnee,1975-T1,1975-T2,1975-T3,1975-T4,1976-T1,1976-T2,1976-T3,1976-T4,1977-T1,...,2022-T2,2022-T3,2022-T4,2023-T1,2023-T2,2023-T3,2023-T4,2024-T1,2024-T2,2024-T3
11,Ensemble,3.2,3.5,3.8,3.9,4,4,3.9,4,4.2,...,7.4,7.2,7.1,7.1,7.2,7.4,7.5,7.5,7.3,7.4


In [33]:
confiance = download_excel_to_dataframe("https://www.insee.fr/en/statistiques/serie/telecharger/xlsx/001587668?ordre=antechronologique&transposition=donneesligne&periodeDebut=10&anneeDebut=1972&periodeFin=11&anneeFin=2024&revision=sansrevisions")
confiance

Le fichier téléchargé est un fichier ZIP.
Fichier Excel non spécifié, utilisation de : serie_001587668_23122024.xlsx
Feuille non spécifiée, chargement de : monthly_values
Feuille 'monthly_values' chargée.


c:\Users\Carla Lucas\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Label,idBank,Last update,Period,2024-11,2024-10,2024-09,2024-08,2024-07,2024-06,...,1973-07,1973-06,1973-05,1973-04,1973-03,1973-02,1973-01,1972-12,1972-11,1972-10
0,Monthly consumer confidence survey - Summary i...,1587668,27/11/2024 08:45,NaN,90.0,93.0,95.0,92.0,91.0,90.0,...,127.0,129.0,131.0,131.0,131.0,130.0,130.0,129.0,128.0,127.0


In [29]:
confiance.drop(columns=columns_to_drop)

None
